This notebook runs a number of queries to check on the status of the TIR database.

In [52]:
import requests,configparser
from IPython.display import display

In [23]:
# Get API keys and any other config details from a file that is external to the code.
config = configparser.RawConfigParser()
config.read_file(open(r'../config/stuff.py'))

In [24]:
# Build base URL with API key using input from the external config.
def getBaseURL():
    gc2APIKey = config.get('apiKeys','apiKey_GC2_BCB').replace('"','')
    apiBaseURL = "https://gc2.mapcentia.com/api/v1/sql/bcb?key="+gc2APIKey
    return apiBaseURL

In [49]:
countITISExact = requests.get(getBaseURL()+"&q=SELECT COUNT(*) AS num FROM tir.tir2 WHERE itis->'itisMatchMethod' LIKE 'ExactMatch%'").json()
print ("Exact ITIS Matches: "+str(countITISExact["features"][0]["properties"]["num"]))

countITISFuzzy = requests.get(getBaseURL()+"&q=SELECT COUNT(*) AS num FROM tir.tir2 WHERE itis->'itisMatchMethod' LIKE 'FuzzyMatch%'").json()
print ("Fuzzy ITIS Matches: "+str(countITISFuzzy["features"][0]["properties"]["num"]))

countITISNot = requests.get(getBaseURL()+"&q=SELECT COUNT(*) AS num FROM tir.tir2 WHERE itis->'itisMatchMethod' LIKE 'NotMatched%'").json()
print ("ITIS Not Matched: "+str(countITISNot["features"][0]["properties"]["num"]))

Exact ITIS Matches: 14799
Fuzzy ITIS Matches: 845
ITIS Not Matched: 2121


In [55]:
countByRank = requests.get(getBaseURL()+"&q=SELECT COUNT(*) as numrank, itis -> 'rank' AS rank FROM tir.tir2 WHERE itis -> 'itisMatchMethod' NOT LIKE 'NotMatched%' GROUP BY itis -> 'rank'").json()
for feature in countByRank["features"]:
    print (feature["properties"]["rank"]+" - "+str(feature["properties"]["numrank"]))

Class - 2
Family - 186
Genus - 53
Order - 24
Phylum - 3
Species - 13552
Subclass - 1
Subfamily - 1
Suborder - 2
Subspecies - 1381
Variety - 439


In [37]:
itisNotMatched = requests.get(getBaseURL()+"&q=SELECT itis -> 'itisMatchMethod' AS notmatched FROM tir.tir2 WHERE itis -> 'itisMatchMethod' LIKE 'NotMatched%' ORDER BY itis -> 'matchMethod'").json()
for feature in itisNotMatched["features"]:
    print (feature["properties"]["notmatched"])

NotMatched:Actinopyga mauritiana
NotMatched:Potamilus alatus
NotMatched:Drosophila neoclavisetae
NotMatched:Rosenus cruciatus
NotMatched:Hibiscus waimeae subsp. hannerae
NotMatched:Polygonum sawatchense ssp. sawatchense g4g5tnrs1s3
NotMatched:Artace cribraria
NotMatched:Aspidoscelis burti
NotMatched:Aspidoscelis exsanguis
NotMatched:Aspidoscelis flagellicauda
NotMatched:Aspidoscelis xanthonota
NotMatched:Asterias forbesii
NotMatched:Stauroneus maunakeensis
NotMatched:Birgus latro
NotMatched:Borodinia missouriensis
NotMatched:Pyganodon subgibbosa
NotMatched:Caecidotea sp. 1
NotMatched:Caecidotea sp. 8
NotMatched:Calidris subruficollis
NotMatched:Brachyranphus marmoratus
NotMatched:Cicinnus melsheimeri
NotMatched:Eutamias cinereicollis
NotMatched:Deschampsia flexuosa
NotMatched:Desmognathus cf. conanti
NotMatched:Diapensia lapponica ssp. lapponica
NotMatched:Euthyonidiella trita
NotMatched:Faronta rubripennis
NotMatched:Flabellorhagidia pecki
NotMatched:Eutrichapion huron
NotMatched:Inci

In [47]:
countByFWSListingStatus = requests.get(getBaseURL()+"&q=SELECT COUNT(*) as numstatus, tess -> 'StatusText' AS status FROM tir.tir2 WHERE exist(tess, 'StatusText') GROUP BY tess -> 'StatusText'").json()
for feature in countByFWSListingStatus["features"]:
    print (feature["properties"]["status"]+" - "+str(feature["properties"]["numstatus"]))

Resolved Taxon - 748
Status Undefined - 60
Original Data in Error - Taxonomic Revision - 2
Species of Concern - 1034
Candidate - 27
Similarity of Appearance to a Threatened Taxon - 11
Proposed Threatened - 10
Threatened - 224
Endangered - 902
Proposed Endangered - 5
Recovery - 24
Original Data in Error - New Information Discovered - 3
Original Data in Error - Erroneous Data - 3
Under Review in the Candidate or Petition Process - 465
Experimental Population, Non-Essential - 41


In [48]:
countByNatureServeGlobalStatus = requests.get(getBaseURL()+"&q=SELECT COUNT(*) as numstatus, natureserve -> 'roundedGlobalStatusRankDescription' AS status FROM tir.tir2 WHERE exist(natureserve, 'roundedGlobalStatusRankDescription') GROUP BY natureserve -> 'roundedGlobalStatusRankDescription'").json()
for feature in countByNatureServeGlobalStatus["features"]:
    print (feature["properties"]["status"]+" - "+str(feature["properties"]["numstatus"]))

Vulnerable - 1651
Apparently Secure - 1841
Presumed Extinct - 19
Possibly Extinct - 114
Secure - 3937
Not Yet Ranked - 862
Critically Imperiled - 2005
Not Applicable - 3
Unrankable - 37
Imperiled - 1367
